# Autokorekta

## Language Model
Prawdopodobieństwo tego, że dana fraza pojawi się jako słowo w angielskim tekście.
Szacujemy prawdopodobieństwo danego słowa licząc wystąpienia każdego słowa w pliku tekstowym zawierającym 1161192 słów.

In [15]:
import nltk
nltk.download('brown')

[nltk_data] Error loading brown: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [21]:
from collections import Counter
words = Counter(nltk.corpus.brown.words())


def p(word, n=sum(words.values())):
    return words[word] / n

## Candidate Model

### Odległość Damerau-Levenshteina

Odległość Damerau-Levenshteina jest miarą odmienności napisów. Jest zdefiniowana jako minimalna liczba operacji potrzebna do przeobrażenia jednego napisu w inny, gdzie operacją może być wstawienie znaku, usunięcie znaku, zastąpienie znaku innym lub przestawienie sąsiednich znaków.

In [31]:
import string
char_set = string.ascii_lowercase


def edits1(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    inserts = [L + c + R for L, R in splits for c in char_set]

    deletes = [L + R[1:] for L, R in splits if R]

    substitutions = [L + c + R[1:] for L, R in splits if R for c in char_set]

    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    
    return set(deletes + transposes + substitutions + inserts)


def edits(word, dist):
    if dist == 1:
        return edits1(word)
    return set(e for generated_word in edits(word, dist-1) for e in edits1(generated_word))


def known(cands):
    return set(w for w in cands if w in words)

## Error Model

Prawdopodobieństwo, że wybrana fraza będzie słowem, które autor chciał wpisać.

Szukamy słowa o jak największym prawdopodobieństwie przy jak najmniejszej odległości od wpisanej frazy. Przyjmujemy, że wszystkie znane słowa o odległości 1 są nieskończenie bardziej prawdopodobne niż znane słowa o odległości 2 i nieskończenie mniej prawdopodobne niż znane słowo o odległości 0.

In [32]:
def candidates(word):
    return known([word]) or known(edits(word, 1)) or known(edits(word, 2)) or known(edits(word, 3))

In [36]:
def autocorrection(word):
    return max(candidates(word), key=p)

## Ocena

In [40]:
assert autocorrection('speling') == 'spelling'
assert autocorrection('lates') == 'later'
assert autocorrection('korrectud') == 'corrected'
assert autocorrection('bycycle') == 'bicycle'
assert autocorrection('inconvient') == 'inconvenient'
assert autocorrection('arrainged') == 'arranged'
assert autocorrection('peotry') == 'poetry'
assert autocorrection('peotryy') == 'poetry'
assert autocorrection('word') == 'word'